In [7]:

from langchain.agents import create_agent
from langchain.tools import tool
import json
from langchain_openai import ChatOpenAI
from pprint import pprint
from langchain.agents.middleware import wrap_tool_call

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

llm = ChatOpenAI(
    model="llama3.2:3b",
    temperature=0,
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

def get_weather(city: str) -> str:
    """Get weather for from langchain_openai import ChatOpenAIa given city."""
    return f"The capital is {city}!"

agent = create_agent(
    model=llm,
    tools=[search],
    system_prompt="You are a helpful assistant",
    middleware=[handle_tool_errors]
)

result = agent.invoke({"input": "What's the capital of France?"})

result_dict = dict(result)
for key, value in result_dict.items():
    print(key, value)

print(result_dict["messages"])

messages = result_dict["messages"]

for message in list(messages):
    print("="*50)
    print(message)



messages [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 53, 'total_tokens': 54, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'llama3.2:3b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-408', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--69d19a19-d2de-4825-be03-45abb45f902e-0', usage_metadata={'input_tokens': 53, 'output_tokens': 1, 'total_tokens': 54, 'input_token_details': {}, 'output_token_details': {}})]
[AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 53, 'total_tokens': 54, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'llama3.2:3b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-408', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--69d19a19-d2de-4825-be03

In [6]:
from dataclasses import dataclass

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy


model = ChatOpenAI(
    model="llama3.2:3b",
    temperature=0,
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

# Define system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response)

print(response['messages'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['messages'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

{'messages': [HumanMessage(content='what is the weather outside?', additional_kwargs={}, response_metadata={}, id='26282c85-a502-42dd-bf98-df8f8741f301'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 343, 'total_tokens': 357, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'llama3.2:3b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-472', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--02d9f541-9bc7-4fc5-99d0-44456e962c27-0', tool_calls=[{'name': 'get_user_location', 'args': {}, 'id': 'call_2jbdmtvl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 343, 'output_tokens': 14, 'total_tokens': 357, 'input_token_details': {}, 'output_token_details': {}}), ToolMessage(content='Florida', name='get_user_location', id='323cd70e-e362-46b3-9773-e0f445f4e568', tool_call_id='call_2jbdmtvl'), AIMessage(content='Looks like we